In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('meds.csv', keep_default_na=False)
print(df.shape)
df.head()

(6198, 12)


,patient_id,encounter_id,encounter_date,patient_sex,patient_age,smoker_status,tag_name,tag_id,tag_time,tag_type1,tag_type2,tag_child_text
0,293,3,2077-07-30,female,71,current,MEDICATION,DOC0,during DCT,ACE inhibitor,NA,NA: ZESTRIL
1,293,3,2077-07-30,female,71,current,MEDICATION,DOC1,after DCT,statin,NA,NA: ZOCOR
2,293,3,2077-07-30,female,71,current,MEDICATION,DOC2,during DCT,aspirin,NA,NA: ASPIRIN; NA: ECASA
3,293,3,2077-07-30,female,71,current,MEDICATION,DOC6,after DCT,ACE inhibitor,NA,NA: ZESTRIL
4,293,3,2077-07-30,female,71,current,MEDICATION,DOC8,after DCT,aspirin,NA,NA: ASPIRIN; NA: ECASA


In [3]:
for row in xrange(df.shape[0]):
    if df.loc[row, 'tag_type2'] != 'NA':
        this = df.loc[row, ]
        this['tag_type1'] = this['tag_type2']
        df = df.append(this)
print(df.shape)

(6223, 12)


In [4]:
df = df.drop(['tag_id', 'tag_time', 'tag_type2', 'tag_child_text', 'tag_name'], 1)

df = df.drop_duplicates()
print(df.shape)
df.head()

(2199, 7)


,patient_id,encounter_id,encounter_date,patient_sex,patient_age,smoker_status,tag_type1
0,293,3,2077-07-30,female,71,current,ACE inhibitor
1,293,3,2077-07-30,female,71,current,statin
2,293,3,2077-07-30,female,71,current,aspirin
5,293,3,2077-07-30,female,71,current,sulfonylureas
7,293,3,2077-07-30,female,71,current,beta blocker


In [5]:
drug_dict = {key:None for key in list(set(df.tag_type1))}
print(drug_dict)

{'niacin': None, 'DPP4 inhibitors': None, 'thienopyridine': None, 'sulfonylureas': None, 'thiazolidinedione': None, 'beta blocker': None, 'ezetimibe': None, 'aspirin': None, 'ARB': None, 'insulin': None, 'nitrate': None, 'statin': None, 'calcium channel blocker': None, 'ACE inhibitor': None, 'metformin': None, 'fibrate': None, 'diuretic': None}


In [6]:
drug_dict['niacin'] = 'HYPERLIPIDEMIA'
drug_dict['ARB'] = 'HYPERTENSION'
drug_dict['diuretic'] = 'HYPERTENSION'
drug_dict['thienopyridine'] = 'CAD'
drug_dict['aspirin'] = 'CAD'
drug_dict['sulfonylureas'] = 'DIABETES'
drug_dict['thiazolidinedione'] = 'DIABETES'
drug_dict['beta blocker'] = 'HYPERTENSION' # also CAD
drug_dict['DPP4 inhibitors'] = 'DIABETES'
drug_dict['ezetimibe'] = 'HYPERLIPIDEMIA'
drug_dict['calcium channel blocker'] = 'HYPERTENSION'
drug_dict['ACE inhibitor'] = 'HYPERTENSION'
drug_dict['insulin'] = 'DIABETES'
drug_dict['nitrate'] = 'HYPERTENSION'  # mostly emergency use
drug_dict['statin'] = 'CAD'
drug_dict['metformin'] = 'DIABETES'
drug_dict['fibrate'] = 'HYPERLIPIDEMIA'

In [7]:
condition_dict ={}

for key, value in drug_dict.items():
    if value not in condition_dict.keys():
        condition_dict[value] = [key]
    else:
        condition_dict[value] = condition_dict[value] + [key]
        
print(condition_dict)

{'HYPERLIPIDEMIA': ['niacin', 'ezetimibe', 'fibrate'], 'CAD': ['thienopyridine', 'aspirin', 'statin'], 'HYPERTENSION': ['beta blocker', 'ARB', 'nitrate', 'calcium channel blocker', 'ACE inhibitor', 'diuretic'], 'DIABETES': ['DPP4 inhibitors', 'sulfonylureas', 'thiazolidinedione', 'insulin', 'metformin']}


In [8]:
df['condition'] = [drug_dict[x] for x in df.tag_type1]

df['condition_medication'] = df['tag_type1']
df = df.drop('tag_type1', 1)

df.head()

,patient_id,encounter_id,encounter_date,patient_sex,patient_age,smoker_status,condition,condition_medication
0,293,3,2077-07-30,female,71,current,HYPERTENSION,ACE inhibitor
1,293,3,2077-07-30,female,71,current,CAD,statin
2,293,3,2077-07-30,female,71,current,CAD,aspirin
5,293,3,2077-07-30,female,71,current,DIABETES,sulfonylureas
7,293,3,2077-07-30,female,71,current,HYPERTENSION,beta blocker


In [9]:
new_df = pd.DataFrame(columns=list(df))

next_row = 0
for next_patient in list(set(df.patient_id)):
    patient_df = df[df.patient_id == next_patient]
    for next_encounter in list(set(patient_df.encounter_id)):
        encounter_df = patient_df[patient_df.encounter_id == next_encounter]       
        for next_condition in list(set(encounter_df.condition)):
            this_group = encounter_df[encounter_df.condition == next_condition]
            
            next_row += 1
            for col in list(new_df):
                new_df.loc[next_row, col] = "; ".join([str(x) for x in set(this_group[col])])
    
new_df.head()

,patient_id,encounter_id,encounter_date,patient_sex,patient_age,smoker_status,condition,condition_medication
1,220,1,2067-05-03,female,55,past,HYPERTENSION,nitrate; beta blocker; calcium channel blocker...
2,220,1,2067-05-03,female,55,past,CAD,statin; thienopyridine; aspirin
3,220,2,2068-12-05,female,37,past,HYPERTENSION,nitrate; beta blocker; calcium channel blocker...
4,220,2,2068-12-05,female,37,past,CAD,statin; thienopyridine; aspirin
5,220,3,2070-12-01,NA,37,past,HYPERTENSION,nitrate; beta blocker; calcium channel blocker...


In [10]:
new_df.to_csv('meds_simplified.csv', index_label=False)